In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 53 kB/s 
     |████████████████████████████████| 199 kB 71.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=4731ad0021376940bb995728fc49a0d413cf7a939e84af4ff13d12ba82153168
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...|2005-10-14 00:00:00|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...| 

In [4]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ58W7SMO911M|          4|            2|          3|   N|                N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
| RATOTLA3OF70O|          5|            2|          2|   N|                N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|                N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|                N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|                N|
|R2XIM9LT335WHE|          4|            3|         10|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|     

In [5]:
vine_reviews_df = vine_df.filter("total_votes>=20")
vine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1AABFZH0J0C0C|          3|           17|         35|   N|                N|
|R3SJYLG07EHNE9|          2|           16|         34|   N|                N|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
|R1KPOIT5WT3NEF|          1|           13|         29|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
| R33IFUWCNYFSO|          2|            4|         45|   N|                N|
| RWMT2MXG3WLDP|          5|            6|         20|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|     

In [6]:
# Create DataFrame where helpful_votes divided by total_votes is equal to or greater than 50%
perc_vine_reviews_df = vine_reviews_df.withColumn('percent_votes',vine_reviews_df['helpful_votes']/vine_reviews_df['total_votes']*100).filter('percent_votes>=50')
perc_vine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|    percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|             80.0|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|67.74193548387096|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|96.42857142857143|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|            100.0|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|87.29641693811075|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|88.88888888888889|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|95.83333333333334|
|R2JSWNTAO83X3V|          5|          14

In [7]:
# Create a DataFrame with all paid reviews (vine == Y)
paid_reviews_df = perc_vine_reviews_df.filter(perc_vine_reviews_df["vine"]=="Y")
paid_reviews_df.show()

+---------+-----------+-------------+-----------+----+-----------------+-------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_votes|
+---------+-----------+-------------+-----------+----+-----------------+-------------+
+---------+-----------+-------------+-----------+----+-----------------+-------------+



In [8]:
unpaid_reviews_df = perc_vine_reviews_df.filter(perc_vine_reviews_df["vine"]=="N")
unpaid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|    percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|             80.0|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|67.74193548387096|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|96.42857142857143|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|            100.0|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|87.29641693811075|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|88.88888888888889|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|95.83333333333334|
|R2JSWNTAO83X3V|          5|          14

In [9]:
five_star_reviews = unpaid_reviews_df.filter('star_rating=5').count()
print(f'There are {five_star_reviews} 5-star reviews.')

n_reviews = unpaid_reviews_df.count()
print(f'There are {n_reviews} total reviews.')

perc_fivestar_reviews = five_star_reviews/n_reviews*100
print(f'{perc_fivestar_reviews}% of reviews have 5 stars.')

There are 242889 5-star reviews.
There are 403807 total reviews.
60.14977452099642% of reviews have 5 stars.
